In [1]:
from keras import backend as K
import cv2 as cv
import numpy as np
import os
import matplotlib.pyplot as plt

/home/bohdan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/bohdan/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def draw_cropped_img(img):
    """
    Draw the borders where the image will be cropped. 
    """
    plt.figure(figsize=(20,10))
    img[103:104,:,0] = 255
    img[:,192:193,0] = 255
    img[:,-65:-64,0] = 255
    img[-10:-9,:,0] = 255
    plt.imshow(img)
    plt.axis('Off')
    
                def preprocess_image(inp_img):
                    """
                    Crop inputed img + make redundant pixels darker
                    Return:
                     - img of size 544x544
                    """
                    img = np.copy(inp_img)
                    img = img[12:,192:-64, :] # crop
                    img[:105,:,:] = 0 # make pixel area dark
                    img[-10:,:,:] = 0 # make pixel area dark

                    return img.astype(np.uint8)

                def preprocess_images(list_of_images):
                    """
                    Preprocess batch of images
                    """
                    return [preprocess_image(img) for img in list_of_images]

def preprocess_mask(mask):
    """
    Crop the mask + set all values into the set of values [0, 1]
    """
    new_mask = np.copy(mask)
    new_mask = new_mask[12:,192:-64, :]
    new_mask[new_mask!=0] = 1
    new_mask = new_mask.astype(np.uint8)
#     new_mask[:,:,1][np.logical_and(new_mask[:,:,0], new_mask[:,:,1])] = 1
    new_mask[:,:,0][np.logical_and(new_mask[:,:,0], new_mask[:,:,1])] = 0 # delete intersection between lumen and tissue
    new_mask[:,:,2][np.logical_and(new_mask[:,:,0], new_mask[:,:,2])] = 0 # delete intersecion between artifact and tissue

    return new_mask

def preprocess_masks(list_of_masks):
    return [preprocess_mask(mask) for mask in list_of_masks]

def save_images(imgs, folder_to_save, filenames):
    if not os.path.exists(folder_to_save):
        os.makedirs(folder_to_save)
    for i in range(len(imgs)):
        cv.imwrite(os.path.join(folder_to_save, filenames[i]), imgs[i])
        
def vis_redundant_pixels(img):
    """
    ImageJ save in masks lots of thrash pixels. But we need only binary values for our Neural
    Networks. So, fuction visualise all this thrash. 
    
    """
    img = np.copy(img)
    img[img == 255] = 0
    img[img != 0] = 255

    print(np.unique(img, return_counts=True))
#     img[img != 255] = 0
    plt.figure(figsize=(20,10))
    plt.imshow(img, cmap='gray')
    plt.axis("Off")

# Loading Data

In [3]:
PATH_TO_X = '/home/bohdan/ultrasound/dataset/norm/x/'
PATH_TO_Y = '/home/bohdan/ultrasound/dataset/norm/y/'

PATH_TO_SAVE_X = '/home/bohdan/ultrasound/dataset/extended_dataset_preprocessed/x_val'
PATH_TO_SAVE_Y = '/home/bohdan/ultrasound/dataset/extended_dataset_preprocessed/y_val'

In [4]:
x_names = os.listdir(PATH_TO_X)
y_names = os.listdir(PATH_TO_Y)

In [ ]:
# # RENAME FILES

# for i in range(len(x_names)):
#     for j in range(len(y_names)):
#         if x_names[i] in y_names[j] and len(x_names[i]) < len(y_names[j]):
#             print(x_names, y_names)
# #             os.rename(os.path.join(PATH_TO_Y,y_names[j]), os.path.join(PATH_TO_Y, x_names[i]))
#             break

# x_names = os.listdir(PATH_TO_X)
# y_names = os.listdir(PATH_TO_Y)

In [7]:
imgs = [cv.imread(PATH_TO_X+'/'+x_names[i]) for i in range (len(x_names))]
# masks = [cv.imread(PATH_TO_Y+'/'+x_names[i])[...,::-1] for i in range (len(y_names))]

# Investigate redundant pixel values 

In [ ]:
# visualise redundant tissue class
for mask in masks:
    vis_redundant_pixels(mask)

# Visualisation of preprocessed pair: Image-Mask

In [ ]:
prep_img = preprocess_image(imgs[0])
prep_mask = preprocess_mask(np.copy(masks[0]))

In [ ]:
np.unique(prep_mask, return_counts=True)

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.imshow(prep_img)
plt.axis("Off")

mask_for_vis = np.copy(prep_mask)
mask_for_vis[mask_for_vis == 1] = 255
plt.subplot(1,2,2)

plt.imshow(mask_for_vis)
plt.axis("Off")

In [ ]:
tissue = prep_mask[:,:,0]
lumen = prep_mask[:,:,1]
artifact = prep_mask[:,:,2]

In [ ]:
for i in preprocess_masks(masks):
    print(np.unique(np.logical_and(i[:,:,0], i[:,:,1]).astype(int), return_counts=True))

# Check wether our classes doesn't intersect

In [ ]:
test_im = np.zeros(preprocess_masks(masks)[0].shape).astype(np.uint8)

In [ ]:
test_im[np.logical_and(preprocess_masks(masks)[0][:,:,0], preprocess_masks(masks)[0][:,:,1])] = 255

In [ ]:
np.unique(test_im, return_counts=True)

In [ ]:
prepr_m = preprocess_masks(masks)

In [ ]:
for a in prepr_m:
    a[a==1] = 255
    plt.figure(figsize=(20,10))
    plt.imshow(a)

# Preprocessing

In [8]:
imgs = preprocess_images(imgs)
# masks = preprocess_masks(masks)

# Saving

In [9]:
# for i in range(len(imgs)):
save_images(imgs, PATH_TO_SAVE_X, x_names)
# save_images(masks, PATH_TO_SAVE_Y, x_names)

In [5]:
PATH_TO_X = '/home/bohdan/ultrasound/dataset/patients/old_105520_65/'
PATH_TO_SAVE_X = '/home/bohdan/ultrasound/dataset/patients/105520_65/'
x_names = os.listdir(PATH_TO_X)
